In [35]:
!pip install -q pymcel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.9/888.9 kB 22.5 MB/s eta 0:00:00


In [36]:
import numpy as np
import pandas as pd
import pymcel as pc
import spiceypy as spy
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [37]:
data = pd.read_table('/content/Enanos.txt')

In [38]:
data_b = pd.read_table('/content/TNOs.txt')

In [40]:
a = data['a'].to_numpy()
e = data['e'].to_numpy()
w = np.deg2rad(data['Peri.'].to_numpy())
inc = np.deg2rad(data['Incl.'].to_numpy())
omega = np.deg2rad(data['Node'].to_numpy())

In [39]:
a_b = data_b['a'].to_numpy()
e_b = data_b['e'].to_numpy()
M = data_b['M'].to_numpy()
w_b = np.deg2rad(data_b['Peri.'].to_numpy())
inc_b = np.deg2rad(data_b['Incl.'].to_numpy())
omega_b = np.deg2rad(data_b['Node'].to_numpy())

In [51]:
def f0_anomalia(M, e):
  N = len(M)
  E = np.zeros(N)

  for i in range(N):
    E[i] = pc.kepler_newton(M[i], e[i], delta = 1e-14)[0]

  f0s = np.arctan2(((1 - e**2)**0.5)*np.sin(E), np.cos(E)-e)
  return f0s

In [42]:
def orbit(a, e, w, inc, omega, f):
  N = len(f)

  r = a*(1 - e**2)/(1 + e*np.cos(f))

  x = np.cos(f)*r
  y = np.sin(f)*r
  z = np.zeros(N)

  M = spy.eul2m(w, inc, omega, 3, 1, 3)

  rvecs_3p = np.column_stack((x, y, z))
  rvecs = rvecs_3p @ M

  return rvecs

In [54]:
N = len(a)
N_b = len(M)
f = np.linspace(0, 2*np.pi, 360)
f0 = f0_anomalia(M, e_b)

bodys = np.zeros((N, 360, 3))
bodys_b = np.zeros((N_b, 3))

names = data['Designation (and name)'].to_numpy()

for i in range(N):
  bodys[i] = orbit(a[i], e[i], w[i], inc[i], omega[i], f)

for i in range(N_b):
  bodys_b[i] = orbit(a_b[i], e_b[i], w_b[i], inc_b[i], omega_b[i], [f0[i]])

In [61]:
fig = go.Figure()

fig.update_layout(
    title = "TNO's Orbit",
    width = 900, height = 675,
    title_x = 0.5)

fig.add_trace(go.Scatter3d(
              x = [0],
              y = [0],
              z = [0],
              mode='markers', name = 'Sun',
              marker = dict(size = 6, color = 'yellow')))

for i in range(N):
  fig.add_trace(go.Scatter3d(
            x = bodys[i, :, 0],
            y = bodys[i, :, 1],
            z = bodys[i, :, 2],
            mode='lines', name = names[i],
            marker = dict(size = 1)))

fig.add_trace(go.Scatter3d(
            x = bodys_b[:, 0],
            y = bodys_b[:, 1],
            z = bodys_b[:, 2],
            mode='markers', showlegend = False,
            marker = dict(size = 1)))

fig.show()